In [8]:
include("network.jl")

build_graph_advanced (generic function with 1 method)

In [9]:
model = build_graph()

13-element Vector{Any}:
 const [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;]
 var wh1
 ┣━ ^ 3×3×1×6 Array{Float64, 4}
 ┗━ ∇ Nothing
 op.?(typeof(conv))
 op.?(typeof(relu))
 op.?(typeof(maxpool))
 op.?(typeof(flatten))
 var wh2
 ┣━ ^ 84×1014 Matrix{Float64}
 ┗━ ∇ Nothing
 op.?(typeof(dense))
 op.?(typeof(relu))
 var wo
 ┣━ ^ 10×84 Matrix{Float64}
 ┗━ ∇ Nothing
 op.?(typeof(dense))
 const [0.0; 0.0; … ; 0.0; 0.0;;]
 op.?(typeof(cross_entropy_loss))

In [10]:
x_train = reshape(train_data.features, 28, 28, 1, :) # insert trivial channel dim
y_train  = onehotmatrix(train_data.targets, 0:9)

x_test = reshape(test_data.features, 28, 28, 1, :) # insert trivial channel dim
y_test = onehotmatrix(test_data.targets, 0:9)

settings = (;
eta = 0.01,
epochs = 3,
batchsize = 100,
)


(eta = 0.01, epochs = 3, batchsize = 100)

In [11]:
global accurate = 0;
global data_count = 0;

In [12]:
test_model(model, x_test, y_test)

ACCURACY: 0.139


In [13]:
train_model(model, x_train, y_train, settings.batchsize, settings.eta, settings.epochs)

 36.009148 seconds (264.62 M allocations: 59.458 GiB, 5.04% gc time, 3.16% compilation time)
Epoch loss: 11.468542353984395
 32.977964 seconds (263.62 M allocations: 59.391 GiB, 4.27% gc time)
Epoch loss: 4.73722714668334
 33.429569 seconds (263.63 M allocations: 59.391 GiB, 4.25% gc time)
Epoch loss: 3.355269542028862


In [14]:
test_model(model, x_test, y_test)

ACCURACY: 0.6882


In [1]:
forward!(model)

UndefVarError: UndefVarError: `forward!` not defined

In [2]:
backward!(model)

UndefVarError: UndefVarError: `backward!` not defined

In [3]:
model[11].inputs[1].output

UndefVarError: UndefVarError: `model` not defined

In [4]:
model[7].output

UndefVarError: UndefVarError: `model` not defined

In [5]:
a = randn(26,26,6);
b = randn(1014,1);
c = randn(84,1);
d = randn(10,84)

10×84 Matrix{Float64}:
 -1.59692     0.119496   -0.233094  …   0.66869   -1.41973     0.590247
  0.496393   -0.971473    1.40272      -0.91225   -1.46724    -0.315003
  2.0568      0.553842   -0.225756     -0.992327  -0.0945981   1.27455
  1.76273    -1.24232     1.8139        0.197804   1.58451     1.18616
  0.105657   -1.24896     0.119251     -0.490409  -1.52854    -0.231467
 -0.0373704  -0.0496929  -0.462907  …  -1.0967    -1.26012     1.02658
  0.882646   -1.19333    -0.480843      1.84618    1.06033    -1.07021
  0.879241   -0.0603558   1.09405       0.15775   -0.928707   -0.295615
  0.904207   -0.306796   -0.644774     -1.72511   -0.110126    0.0718428
 -1.9446     -0.435578    0.236105      2.09028   -0.0333993  -0.608892

In [6]:
uniform_rand(3,3,1,6)

UndefVarError: UndefVarError: `uniform_rand` not defined

In [7]:
using BenchmarkTools

In [8]:
function relu1(x)
    for i in eachindex(x)
        x[i] = max(0, x[i])
    end
    return x
end


relu1 (generic function with 1 method)

In [9]:
r1 = max.(a, 0)


26×26×6 Array{Float64, 3}:
[:, :, 1] =
 0.0        0.0       0.590243   0.181446   …  0.314975  0.377915  0.0
 0.0        0.52713   1.37331    1.49359       0.0       1.79771   0.0
 0.683476   1.11443   0.0        1.28406       0.318642  0.194292  0.251619
 1.54985    0.237073  1.16829    2.23983       0.0       0.522435  0.169968
 0.0        0.285383  0.782714   0.0           0.998153  0.0       0.0
 0.0        0.232521  0.49587    0.0        …  1.19066   0.175274  1.28423
 0.0        0.453986  0.334899   0.0           0.708452  0.0       0.158727
 0.0        1.02543   0.573374   0.0           0.372952  0.937851  0.60848
 0.0        0.0       0.507744   0.0           0.0       0.0       0.0
 0.0        0.393748  0.0147179  2.5296        0.0       0.0       0.0
 ⋮                                          ⋱                      ⋮
 0.570814   0.0       0.0        0.244366      0.559015  0.0       0.622702
 0.0        0.0       0.0        0.0           0.0       0.880486  0.590458
 0.0919

In [10]:
r2 = relu1(a)

26×26×6 Array{Float64, 3}:
[:, :, 1] =
 0.0        0.0       0.590243   0.181446   …  0.314975  0.377915  0.0
 0.0        0.52713   1.37331    1.49359       0.0       1.79771   0.0
 0.683476   1.11443   0.0        1.28406       0.318642  0.194292  0.251619
 1.54985    0.237073  1.16829    2.23983       0.0       0.522435  0.169968
 0.0        0.285383  0.782714   0.0           0.998153  0.0       0.0
 0.0        0.232521  0.49587    0.0        …  1.19066   0.175274  1.28423
 0.0        0.453986  0.334899   0.0           0.708452  0.0       0.158727
 0.0        1.02543   0.573374   0.0           0.372952  0.937851  0.60848
 0.0        0.0       0.507744   0.0           0.0       0.0       0.0
 0.0        0.393748  0.0147179  2.5296        0.0       0.0       0.0
 ⋮                                          ⋱                      ⋮
 0.570814   0.0       0.0        0.244366      0.559015  0.0       0.622702
 0.0        0.0       0.0        0.0           0.0       0.880486  0.590458
 0.0919

In [11]:
x = randn(28,28,1)
w = randn(3,3,1,6)
g = randn(26,26,6,1)

26×26×6×1 Array{Float64, 4}:
[:, :, 1, 1] =
  0.300281  -2.13027     1.45273   …  -0.438623   -1.2835     -0.441807
  0.244671  -2.03169     1.60355      -1.06383     0.240266   -2.21688
 -0.504992   1.25219    -0.435919      0.135189   -0.693574   -0.150326
  3.21575   -0.514418   -1.49007       0.298719    0.089       1.05784
 -0.799657   1.25911    -0.617887     -0.649534   -0.838202    0.457178
  1.05554    2.50779     0.831416  …   1.19258    -0.66654    -0.168416
 -0.177217   0.439452   -0.30382       0.135214    0.478825   -1.14519
 -0.689479   0.11143    -0.88436      -0.373126    0.806133   -0.724205
 -1.67948    0.313526   -1.16315       1.68738     1.01759    -1.14782
  0.299225   0.800883    0.972526      0.0539452   0.399215    1.18162
  ⋮                                ⋱                           ⋮
  0.198053   0.236063   -1.78809       0.246227   -0.0309108   1.35333
 -0.77692   -0.0295436   1.77255       1.136      -0.412723    0.565092
  3.97119   -0.948387   -0.142236